## Chapter 15: Calculus and automatic differentiation 

# 15.5  The forward mode of automatic differentiation

In the previous Section we saw how representing a function via its computation graph allows us to evaluate it at by traversing the graph in a forward direction, from left to right, recursively evaluating local sets of children-parent nodes.  Here we will see how the computation graph also allows us to construct and evaluate a function's gradient by sweeping forward through the function's computation graph recursively, once again in sets of children-parent nodes.  The recursive aspect of this procedure, which is called the *forward mode of automatic differentiation*, makes it easily programmable and allows for the transfer of the tedius chore of gradient computation to a computer program which makes gradient computation faster and more reliable (than when performed by hand).

In [1]:
# This code cell will not be shown in the HTML version of this notebook
#imports from custom library
import sys
sys.path.append('../../')
import autograd.numpy as np
import matplotlib.pyplot as plt
from mlrefined_libraries import calculus_library as calclib
from mlrefined_libraries import basics_library as baslib

%matplotlib notebook
from matplotlib import rcParams
rcParams['figure.autolayout'] = True

The first step to the forward mode of automatic differentiation is to *compute the derivative with respect to the function's input at each node in the computation graph* of a function.  So, in the case of a single-input function, this means that we first compute the form of $\frac{\mathrm{d}}{\mathrm{d}w}$ at *each node in the graph*.  As with the function evaluation itself this is performed *recursively* starting at the input node and sweeps forward from left to right in sets of child-parent nodes.

Take for example the function $g(w) = \text{tanh}(w)\text{cos}(w) + \text{log}(w)$ whose computation graph was illustrated in the previous Section.  To compute the form of the derivative $\frac{\mathrm{d}}{\mathrm{d}w}{g(w)}$ we begin at the input node and compute $\frac{\mathrm{d}}{\mathrm{d}w}{w}$, which trivially is always equal to $1$.  We then move to the parents of this node, that is nodes $a$, $b$, and $c$, and compute their derivatives with respect to the input - that is $\frac{\mathrm{d}}{\mathrm{d}w}{a}$, $\frac{\mathrm{d}}{\mathrm{d}w}{b}$, and $\frac{\mathrm{d}}{\mathrm{d}w}{c}$.  This is illustrated figuratively in the left panel below, with the child node highlighted in blue and the parents in red.

<figure>
<img src="../../mlrefined_images/calculus_images/compgraph_2.png" width="100%" >  
<figcaption> <strong>Figure 1:</strong> <em> An example of forward mode derivative computation of a single input function.  See text for further details.
</em>
</figcaption>
</figure>

Computing each parent-child derivative requires only the value computed at their children, and the derivative rules for elementary functions and operations described in Section 15.2.  In other words, we have for each parent node that their derivatives with respect to the input $w$ are given as
    
\begin{array}
\
\frac{\mathrm{d}}{\mathrm{d}w}a(w) = (1 - \text{tanh}^2(w))\\
\frac{\mathrm{d}}{\mathrm{d}w}b(w)  = -\text{sin}(w) \\\
\frac{\mathrm{d}}{\mathrm{d}w}c(w) = \frac{1}{w}. \\
\end{array}

Notice that in order to compute the derivative at each parent *we only need the elementary derivative rules and the information contained in the child node*.  Here the information from the child node at $w$ was the trivial derivative $\frac{\mathrm{d}}{\mathrm{d}w}w = 1$. 

With these derivatives with respect to $w$ computed we move upwards to the next parents in the graph , where we will see the same pattern emerge, seeking to derive derivatives of these nodes with respect to $w$.  Examining the graph we can see that the parent of $a$ and $b$ is $d$, and the parent of $c$ is $e$.  Since the children of $d$ have already computed derivatives, we compute its derivative next.  

To compute $\frac{\mathrm{d}}{\mathrm{d}w}d(a,b)$ (illustrated in the middle panel of the figure above) we use the *product rule* (from Table 2 in Section 15.2) which gives

\begin{equation}
\frac{\mathrm{d}}{\mathrm{d}w}d(a,b) = \left(\frac{\mathrm{d}}{\mathrm{d}w}a(w)\right)\times b(w)  + a(w) \times \left(\frac{\mathrm{d}}{\mathrm{d}w}b(w)\right)
\end{equation}

Because we have already computed the form of the derivatives of $a$ and $b$ with respect to $w$, we need only  compute the parent-child derivatives $\frac{\mathrm{d}}{\mathrm{d}a}d(a,b)$ and $\frac{\mathrm{d}}{\mathrm{d}b}d(a,b)$.  Both are given by elementary derivative function rules (in Table 1 of Section 15.2), and thus we have

\begin{array}
\
\frac{\mathrm{d}}{\mathrm{d}a}d(a,b) = b = \text{cos}(w)\\
\frac{\mathrm{d}}{\mathrm{d}b}d(a,b) = a = \text{tanh}(w).\\
\end{array}

Thus the form of the entire derivative can be computed explicitly at node $d$ as

\begin{equation}
\frac{\mathrm{d}}{\mathrm{d}w}d(a,b) = (1 - \text{tanh}^2(w))\times \text{cos}(w)  + \text{tanh}(w) \times (-\text{sin}(w)).
\end{equation}

Now that we have resolved the derivative at $d$ we can work on the final parent node $e$ (illustrated in the right panel of the figure above), which is a parent of both $d$ and $c$.  Since $e$ is defined as the sum of $d$ and $c$ its derivative with respect to $w$ is written as 

\begin{equation}
\frac{\mathrm{d}}{\mathrm{d}w}e(d,c) = \frac{\mathrm{d}}{\mathrm{d}w}d(a,b) + \frac{\mathrm{d}}{\mathrm{d}w}c(w)
\end{equation}

We have already computed both derivatives on the right side above, so plugging both in we have our desired derivative 

\begin{equation}
\frac{\mathrm{d}}{\mathrm{d}w}g(w) = \frac{\mathrm{d}}{\mathrm{d}w}e(d,c) = (1 - \text{tanh}^2(w))\times \text{cos}(w)  + \text{tanh}(w) \times (-\text{sin}(w)) + \frac{1}{w}.
\end{equation}

Having computed the form of the derivative at each node of the graph we can now evaluate both the function *and* its derivative by *sweeping forward* throught the computation graph (precisely as shown for just the function evaluation in the previous Section).  This is why the method is called the *forward mode* of automatic differentation, as all computation is done moving *forward* through the computation graph.  This is illustrated below for the input value $w = 1.5$ where both the function an derivative evaluation $g(1.5)$ and $\frac{\mathrm{d}}{\mathrm{d}w}g(1.5)$ are computed together traversing forward through the computation graph.

<figure>
<img src="../../mlrefined_images/calculus_images/compgraph_1.png" width="100%" >  
<figcaption> <strong>Figure 3:</strong> <em> Once the form of the derivative has been computed at each node, the derivative can be evaluated with the function moving forward through the computation graph.  See text for further details.

NEW IMAGE NEEDED
</em>
</figcaption>
</figure>

This same procedure can be used for multi-input functions as well, except now the first step now requires we *compute the form of the gradient at each node in graph*.  Below we illustrate how this is done using the quadratic $g\left(\mathbf{w}\right) = w_1^2+w_2^2$ whose computation graph was drawn in the previous Section.

<figure>
<img src="../../mlrefined_images/calculus_images/compgraph_4.png" width="80%" >  
<figcaption> <strong>Figure 4:</strong> <em> An example of gradient computation via the forward mode of a multi-input function using its computation graph.  See text for further details.
CHANGE [0,1] / [1,0] TO PARTIALS AT ROOT NODES
</em>
</figcaption>
</figure>

Following the pattern set forth with the single input example above, we begin by computing the gradient at each input node.  That is, we compute $\nabla w_1$ and $\nabla w_2$.  Note here since $\nabla = \begin{bmatrix}\frac{\partial}{\partial w_{1}} \\
\frac{\partial}{\partial w_{2}}
\end{bmatrix}$ that we trivially have $\nabla w_1 = \begin{bmatrix} 1 \\ 0
\end{bmatrix}$ and $\nabla w_2 = \begin{bmatrix} 0 \\ 1
\end{bmatrix}$.
 

We then move to the parents of our input nodes beginning say at node $a$ (as illustrated in the left panel below), where we compute the *gradient* at each node on the left, i.e., the partial derivatives of $a\left(w_1\right)=w_1^2$ with respect to both $w_1$ and $w_2$ as

\begin{equation}
\frac{\partial}{\partial w_{1}}a = 2w_1 \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,
\frac{\partial}{\partial w_{2}}a = 0.
\end{equation}


Similarly, we can compute the partial derivatives of $b\left(w_1\right)=w_2^2$ with respect to $w_1$ and $w_2$ (also illustrated in the left panel below), as 

\begin{equation}
\frac{\partial}{\partial w_{1}}b = 0 \,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,
\frac{\partial}{\partial w_{2}}b = 2w_2.
\end{equation}

With the form of the gradient computed at nodes $a$ an $b$ we can finally compute the gradient at their parent node - $c$.  The partial derivatives of $c(a,b)=a+b$ with respect to $w_1$ and $w_2$ are computed via the chain rule (as illustrated in the right panel below) and employ the derivative forms computed at each child as

\begin{equation}
\frac{\partial}{\partial w_{1}}c = \frac{\partial}{\partial a}c\,\frac{\partial}{\partial w_{1}}a+\frac{\partial}{\partial b}c\,\frac{\partial}{\partial w_{1}}b = 1 \times 2w_1 + 1 \times 0 = 2w_1\\
\frac{\partial}{\partial w_{2}}c = \frac{\partial}{\partial a}c\,\frac{\partial}{\partial w_{2}}a+\frac{\partial}{\partial b}c\,\frac{\partial}{\partial w_{2}}b = 1 \times 0 + 1 \times 2w_2 = 2w_2  
\end{equation}

Together this gives us the form of the gradient at node $c$, and we can now evaluate the function $g$ and its gradient $\nabla g$ as shown with the single input example previously (recursively from left to right moving from child-parent node sets across the computation graph).